# QuAD - Structured data from text with regex - part 3

**Author:** *[Andrew Gibson](https://github.com/andrewresearch)* - **Updated:** *18 Mar 2020*

This Quick Analytics Demo (QuAD) shows how to use regular expressions (regex) to obtain structured data from unstructured text.

**[Part 1](QuAD%20-%20Structured%20data%20from%20text%20with%20regex%20-%20part%201.ipynb)** covers extracting characters from the beginning of each line in a string and searching for a specific pattern using regex.

**[Part 2](QuAD%20-%20Structured%20data%20from%20text%20with%20regex%20-%20part%202.ipynb)** covers splitting strings using regex.

**[Part 3](QuAD%20-%20Structured%20data%20from%20text%20with%20regex%20-%20part%203.ipynb)** covers the creation of dataframes and JSON from data extracted from strings using regex.

---

### Previously...

Our task was to obtain the following data from a given text.

| Date | Name | Food eaten |
| --- | --- | --- |
| 2020.03.18 | Andrew | avocado |
| 2020.03.19 | Catarina | coconut |
| 2020.03.19 | Prime Minister | pineapple mousse |

In Part 1, we extracted the date from the beginning of lines of text (where there was a date to extract).

In Part 2, we extracted the names and foods by splitting the remainder of each line based on a regex that could cope with different eat words.

In [1]:
import re

# the text which we want to obtain data from
text = """
*** Start ***
2020.03.18 Andrew ate avocado
2020.03.19 Catarina eats coconut
2020.03.19 Prime Minister eating pineapple mousse
*** End ***
"""
# split into list of lines
lines = text.split('\n')

# loop over lines and match dates and put in new list
for line in lines:
    print(line)
    match = re.search(r"^[0-9]{4}\.[0-9]{2}\.[0-9]{2}", line)
    if match:
        print("\tDate: ",match.group())
        eatline = line[11:] # substrings for lines that match
        splitline = re.split(r"\s[a-z]+\s",eatline)
        print("\tName: ",splitline[0]) # First element of splitline is name
        print("\tFood: ",splitline[1]) # Second element of splitline is food
        print() # Empty line to make easy to read
    else:
        print("--- not a match ---")
        print()


--- not a match ---

*** Start ***
--- not a match ---

2020.03.18 Andrew ate avocado
	Date:  2020.03.18
	Name:  Andrew
	Food:  avocado

2020.03.19 Catarina eats coconut
	Date:  2020.03.19
	Name:  Catarina
	Food:  coconut

2020.03.19 Prime Minister eating pineapple mousse
	Date:  2020.03.19
	Name:  Prime Minister
	Food:  pineapple mousse

*** End ***
--- not a match ---


--- not a match ---



### Making the data useful for further analysis

While the previous example showed that we could extract the data, it didn't provide it in an easy to use format. 

If we want to make further analysis easy, we need to take our data and put it into an appropriate **Data Structure**.

The two most common data structures we will use are **DataFrames** and **JSON**.

### Extracting to JSON

If we look carefully at the code above, we see that we already have name value pairs, like what we would expect to see in a JSON object.

```javascript
{ 
    "Date": "2020.02.02",
    "Name": "My Name",
    "Food": "my food"
}
```

The Python equivalent is called a dictionary. We can create a dictionary and add name value pairs like this...

In [2]:
# create an empty dictionary
my_dict = {}

# add name-value pairs
my_dict["Date"] = "2020.02.02"
my_dict["Name"] = "My Name"
my_dict["Food"] = "my food"

# view the dictionary
my_dict

{'Date': '2020.02.02', 'Name': 'My Name', 'Food': 'my food'}

This is a Python dictionary, but we can turn this into a JSON string by using the json library...

In [3]:
import json

In [4]:
json.dumps(my_dict)

'{"Date": "2020.02.02", "Name": "My Name", "Food": "my food"}'

Now we can use this json approach to modify our code from part 2...

In [5]:
# create a list to put our dictionaries in
new_list = []

# loop over lines and match dates and put in new list
for line in lines:
    match = re.search(r"^[0-9]{4}\.[0-9]{2}\.[0-9]{2}", line)
    if match:
        # create a new dictionary to put this match in
        new_dict = {}
        # now add the name value pairs
        new_dict["Date"] = match.group()
        eatline = line[11:] # substrings for lines that match
        splitline = re.split(r"\s[a-z]+\s",eatline)
        new_dict["Name"] = splitline[0] # First element of splitline is name
        new_dict["Food"] = splitline[1] # Second element of splitline is food
        new_list.append(new_dict) # Add the dict to the list
    
new_list

[{'Date': '2020.03.18', 'Name': 'Andrew', 'Food': 'avocado'},
 {'Date': '2020.03.19', 'Name': 'Catarina', 'Food': 'coconut'},
 {'Date': '2020.03.19', 'Name': 'Prime Minister', 'Food': 'pineapple mousse'}]

In [6]:
# JSON string version
json.dumps(new_list)

'[{"Date": "2020.03.18", "Name": "Andrew", "Food": "avocado"}, {"Date": "2020.03.19", "Name": "Catarina", "Food": "coconut"}, {"Date": "2020.03.19", "Name": "Prime Minister", "Food": "pineapple mousse"}]'

### Extracting to a DataFrame

In order to use pandas dataframes, we need to import the pandas library. It is common in Python to abreviate pandas as **pd**.

In [7]:
import pandas as pd

Before extracting our data into a dataframe, we need to understand what is required to (a) create a new dataframe (b) append data to that dataframe.

In [9]:
df = pd.DataFrame(columns=['Date','Name','Food'])

In [10]:
df

,Date,Name,Food


pandas expects a dictionary when adding to the dataframe, as it needs to know which values should be put with which columns. Also, if we don't have explicit index values, we need to tell pandas to ignore the index.

In [11]:
dict_to_add = {'Date':'2020.03.15','Name':'Test Name','Food':'test food'}
df = df.append(dict_to_add,ignore_index=True)

In [12]:
df

,Date,Name,Food
0,2020.03.15,Test Name,test food


Notice that because we are adding a dictionary to the dataframe, this is very similar to our JSON code where we were adding a dictionary to a list.

We can therefore modify the code above to add to a dataframe...

In [13]:
# create an empty dataframe
df = pd.DataFrame(columns=['Date','Name','Food'])

# loop over lines and match dates and put in new list
for line in lines:
    match = re.search(r"^[0-9]{4}\.[0-9]{2}\.[0-9]{2}", line)
    if match:
        # create a new dictionary to put this match in
        new_dict = {}
        # now add the name value pairs
        new_dict["Date"] = match.group()
        eatline = line[11:] # substrings for lines that match
        splitline = re.split(r"\s[a-z]+\s",eatline)
        new_dict["Name"] = splitline[0] # First element of splitline is name
        new_dict["Food"] = splitline[1] # Second element of splitline is food
        df = df.append(new_dict,ignore_index=True) # Add the dict to the df
    
df

,Date,Name,Food
0,2020.03.18,Andrew,avocado
1,2020.03.19,Catarina,coconut
2,2020.03.19,Prime Minister,pineapple mousse


That comples our QuAD showing how to extract data from text using regex and put that data into JSON and DataFrame data structures.